In [ ]:
from nbdev import *
%nbdev_default_export architectures

Cells will be exported to pct.architectures,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# Architectures

> Classes to create hierachy architectures from array configurations.

In [ ]:
%nbdev_hide
import sys
sys.path.append("..")

In [ ]:
%nbdev_export
import gym
import random
import numpy as np

import json
import os
from abc import ABC, abstractmethod
from pct.hierarchy import PCTHierarchy
from pct.nodes import PCTNode
from pct.functions import WeightedSum

In [ ]:
%nbdev_export
class BaseArchitecture(ABC):
    "Base class of an array architecture. This class is not used direclty by developers, but defines the functionality common to all."
    def __init__(self, name, config, env, inputs):
        self.config = config      
        self.env = env
        self.inputs=inputs
        self.hpct = PCTHierarchy()
        for input in inputs:
            self.hpct.add_preprocessor(input)
        
    def __call__(self):
        level0config = self.config['level0']
        print(level0config)
        level0=self.configure_zerothlevel()
   
    def get_hierarchy(self):
        return self.hpct
        
 

In [ ]:
%nbdev_export
class ProportionalArchitecture(BaseArchitecture):
    "Proportional Architecture"
    def __init__(self, name="proportional", config=None, env=None, inputs=None, **cargs):
        super().__init__(name, config, env, inputs)



    def configure_zerothlevel(self):
        inputsIndex=0
        outputsIndex=1
        actionsIndex=2

        config=self.config['level0']
        level=0
        numInputs= len(self.inputs)
        columns = len(config[inputsIndex][0])
        #print(config[0][0])
        #print(columns)

        # create nodes
        for column in range(columns):
            node = PCTNode(build_links=True, mode=1)
            # change names
            node.get_function("perception").set_name(f'wsPL{level}C{column}')
            node.get_function("reference").set_name(f'wsRL{level}C{column}')
            node.get_function("comparator").set_name(f'CL{level}C{column}')
            node.get_function("output").set_name(f'pOL{level}C{column}')

            weights=[]
            # configure perceptions
            for inputIndex in range(numInputs):
                node.get_function("perception").add_link(self.inputs[inputIndex])
                weights.append(config[inputsIndex][inputIndex][column])
            node.get_function("perception").weights=np.array(weights)

            # configure outputs
            node.get_function("output").set_property('gain', config[outputsIndex][column])

            self.hpct.add_node(node, level, column)

        # configure actions
        numActions = len(config[actionsIndex])
        numColumnsThisLevel = len(config[outputsIndex])
        for actionIndex in range(numActions):
            action = WeightedSum(weights=config[actionsIndex][actionIndex], name=f'action{actionIndex+1}')
            for column in range(numColumnsThisLevel):
                action.add_link(f'pOL{level}C{column}')
            self.hpct.add_postprocessor(action)
            self.env.add_link(action)




In [ ]:
# move to epct
from epct.configs import ProportionalConfiguration
from pct.environments import PendulumV0

In [ ]:
pen = PendulumV0(render=True, verbose=True)
inputs=[2, 3]
num_actions=1

pc = ProportionalConfiguration(num_inputs=len(inputs), num_actions=num_actions, grid=[2, 2], references=[1])
config = pc()
pa = ProportionalArchitecture(config=config, env=pen, inputs=inputs)
pa()

hpct = pa.get_hierarchy()
hpct.summary()
print(json.dumps(hpct.get_config(), indent=1))



2 2
[[[1, 1], [1, 1]], [62.964989144042704, 58.73258054599337], [[0, 0]]]
pcthierarchy PCTHierarchy
**************************
PRE: None
Level 0
pctnode PCTNode
----------------------------
REF: wsRL0C0 WeightedSum | weights [1. 1. 1.] | 0 
PER: wsPL0C0 WeightedSum | weights [1 1] | 0 | links  

AttributeError: 'int' object has no attribute 'get_name'

In [ ]:
pen.close()

In [ ]:
notebook2script()